In [1]:
%pylab

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib as mpl
import matplotlib.animation as animation

import copy

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
Xmin, Xmax, Npoints_x = 0.,2.,10
Ymin, Ymax, Npoints_y = 0.,2.,10
Zmin, Zmax, Npoints_z = 0.,2.,10

Tmin, Tmax, Npoints_t = 0.,4.,600

x = np.linspace(Xmin, Xmax, Npoints_x)
y = np.linspace(Ymin, Ymax, Npoints_y)
z = np.linspace(Zmin, Zmax, Npoints_z)

t = np.linspace(Tmin, Tmax, Npoints_t)

deltat = (Tmax-Tmin)/(Npoints_t-1)
deltax = (Xmax-Xmin)/(Npoints_x-1)
deltay = (Ymax-Ymin)/(Npoints_y-1)
deltaz = (Zmax-Zmin)/(Npoints_z-1)


v = 6.0

lambda_ = v*deltat/deltax
mu_ = v*deltat/deltay
nu_ = v*deltat/deltaz

print(deltat, deltax, lambda_, mu_, nu_)

0.00667779632721202 0.2222222222222222 0.18030050083472454 0.18030050083472454 0.18030050083472454


In [3]:
# Definimos condicion inicial

def ui(x,y,z):
    return np.sin(np.pi*x)*np.cos(np.pi*y)*np.sin(np.pi*z)

# Definimos el campo de la onda

u = np.zeros((Npoints_x,Npoints_y, Npoints_z))

for i in range(len(x)):
    for j in range(len(y)):
        for k in range(len(z)):
            u[i,j,k] = ui(x[i],y[j],z[k])

#print(u)

In [4]:
u_pasado = u.copy()
u_presente = u.copy()
u_futuro = u.copy()

In [5]:

U = np.zeros((Npoints_t, Npoints_x, Npoints_y, Npoints_z))

gamma = 3.*deltat

for l in range(len(t)):
    
    # Aca ajustamos fronteras moviles
    #u_futuro[:,:,-1] = 2.0*np.sin(np.pi*t[l])
    
    for i in range(1,len(x)-1):
        for j in range(1,len(y)-1):
            for k in range(1,len(z)-1):
                u_futuro[i,j,k] = 2.*(1-lambda_**2-mu_**2-nu_**2)*u_presente[i,j,k] \
                + lambda_**2*(u_presente[i+1,j,k] + u_presente[i-1,j,k]) \
                + mu_**2*(u_presente[i,j+1,k]+u_presente[i,j-1,k]) \
                + nu_**2*(u_presente[i,j,k+1]+u_presente[i,j,k-1]) \
                - u_pasado[i,j,k] \
                - gamma*u_presente[i,j,k] + gamma*u_pasado[i,j,k]

    U[l,:,:,:] = u_futuro
    
      # Actualizacion
    u_pasado=copy.deepcopy(u_presente)
    u_presente=copy.deepcopy(u_futuro)   


In [10]:
# Zona de animacion

Nframes = len(t) 
fig = plt.figure()
ax = fig.add_subplot(1,1,1, projection = '3d',elev = 30, azim = 50)

def init():
    ax.set_xlim3d(Xmin, Xmax)
    ax.set_ylim3d(Ymin, Ymax)
    ax.set_zlim3d(Zmin, Zmax)

    ax.set_xlabel('$x$', fontsize=10)
    ax.set_ylabel('$y$', fontsize=10)
    ax.set_zlabel('$z$', fontsize=10)  

def animate(i):
    
    plot = ax.clear()
    init()
    
    x_, y_, z_ = np.meshgrid(x, y, z, indexing='ij') + U[i]
    
    plot = ax.scatter3D(x_,y_,z_, marker='.', color='b', alpha=0.6)
    fake2Dline = mpl.lines.Line2D([0],[0], linestyle='none', c='b', marker='o')
    plot = ax.legend([fake2Dline], [r'u(%.2f,x,y,z)' %(t[i])], numpoints=1)
    return plot

Animation = animation.FuncAnimation(fig, animate, frames=Nframes, init_func=init)   

In [9]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=1800 )
#Animation.save( 'Onda3D.mp4', writer=writer )